# Proof of Concept
This PoC is based on [Model Weight Theft With Just Noise Inputs](https://arxiv.org/abs/1912.08987).

In [4]:
!pip -qq install lightning

In [5]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

loaders = {}

with np.load("mnist.npz") as f:
    for split in ["train", "test"]:
        X = torch.from_numpy(f[f"x_{split}"]).float()
        y = torch.from_numpy(f[f"y_{split}"]).long()

        X = X / 255.0
        y = F.one_hot(y, num_classes=10).float()

        loaders[split] = DataLoader(
            dataset=TensorDataset(X, y),
            batch_size=1000,
            num_workers=2,
            shuffle=True,
            pin_memory=True,
        )

In [14]:
import torch.nn as nn
import lightning.pytorch as pl

# VICTIM MODEL - Completely blackboxed as can be seen in later code blocks.
class CNN(pl.LightningModule):

    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(1, 2, kernel_size=3),  # (B 2 26 26)
            nn.GELU(),
            nn.MaxPool2d(2),  # (B 2 13 13)
            nn.Flatten(),  # (B 338)
            nn.Linear(338, 2),
            nn.GELU(),
            nn.Linear(2, 10),
        )

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

    def forward(self, input):
        return self.cnn(input.unsqueeze(1))

    def training_step(self, batch, batch_idx):
        return self._step(batch, split="train")

    def validation_step(self, batch, batch_idx):
        return self._step(batch, split="val")

    def test_step(self, batch, batch_idx):
        return self._step(batch, split="test")

    def _step(self, batch, split):
        inputs, probs = batch
        logits = self(inputs)

        loss = F.cross_entropy(logits, probs)
        acc = (logits.argmax(dim=-1) == probs.argmax(dim=-1)).float().mean()

        log_kwargs = dict(batch_size=inputs.shape[0], prog_bar=True)
        self.log(f"{split}/loss", loss,  **log_kwargs)
        self.log(f"{split}/acc", acc, **log_kwargs)

        return loss

In [15]:
trainer = pl.Trainer(
    accelerator="gpu",
    callbacks=None,
    enable_checkpointing=False,
    logger=False,
    max_epochs=20,
    log_every_n_steps=5,
    enable_progress_bar=True,
    use_distributed_sampler=False,
    check_val_every_n_epoch=1,
)

model = CNN()
trainer.fit(model=model, train_dataloaders=loaders["train"])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name | Type       | Params
------------------------------------
0 | cnn  | Sequential | 728   
------------------------------------
728       Trainable params
0         Non-trainable params
728       Total params
0.003     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name | Type       | Params
-----------------

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [16]:
trainer.test(model=model, dataloaders=loaders["test"])

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.7373999953269958     │
│         test/loss         │     0.858664870262146     │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 0.858664870262146, 'test/acc': 0.7373999953269958}]

In [17]:
with torch.no_grad():
    X_rand = [
        (torch.rand(6000, 28, 28) < (p + 0.01)).float()
        for p in torch.linspace(0, 0.9, 10)
    ]
    X_rand = torch.cat(X_rand, dim=0)
    # Unable to replicate results with Gaussian noise, did not test Ising.
    y_rand = F.softmax(model(X_rand), dim=-1)

rand_loader = DataLoader(
    dataset=TensorDataset(X_rand, y_rand),
    batch_size=1000,
    num_workers=2,
    shuffle=True,
    pin_memory=True,
)

In [18]:
trainer = pl.Trainer(
    accelerator="gpu",
    callbacks=None,
    enable_checkpointing=False,
    logger=False,
    max_epochs=200,
    log_every_n_steps=5,
    enable_progress_bar=True,
    use_distributed_sampler=False,
    check_val_every_n_epoch=1,
)

attack = CNN()
trainer.fit(
    model=attack,
    train_dataloaders=rand_loader,
    val_dataloaders=loaders["test"],
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name | Type       | Params
------------------------------------
0 | cnn  | Sequential | 728   
------------------------------------
728       Trainable params
0         Non-trainable params
728       Total params
0.003     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name | Type       | Params
-----------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=200` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.


In [19]:
trainer.test(attack, dataloaders=loaders["test"])

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.7285000085830688     │
│         test/loss         │     0.892433226108551     │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 0.892433226108551, 'test/acc': 0.7285000085830688}]